*Rence, mao ni ang example sa data encoding,  there are things you need to do, kailangan nimo e butang sa dataframe ang kaning mga data nga ge input sa user, such as age, gender, etc., after ana, kailangan pana nimo ipang encode since the model is trained using the encoded inputs, naa koy ge provide sa ubos na function na magamit nimo.*

*but before nimo na gamiton na function specifically kaning <code>encode_features</code>, dapat tanan mga string values anang dataframe where ge store nimo ang data is naka capitalize, makita nimo na ang <code>user_input_df</code>, tanan strings naka capitalize, because ang kanang algorithm kay designed para lang sa capiralized na values such as MALE, LIGHT YELLOW and etc., so dapat e capitalize na nimo sya before nimo sya e butang sa dataframe. you can use this one <code>"sample text".upper()</code> >>> 'SAMPLE TEXT'*

*also dapat, dataframe rajud ang pwede na e pass nimo sa <code>encode_features</code>, and make sure, isa rajud ka ROW and dataframe, if mag input nasad si user og another data nasad, dapat ma clear jud ang dataframe because again, ang algorithm kay designed for isa ra ka row.*

- *Make sure lang jud na tanan inputs kay dili null by making input boxes nga necessary nga naay value, otherwise, mag error sa feature encoding*
- *Check the data types na gina pang input sa user sa pH, Specific Gravity kay pwede sila mag input og string ata danha resulting to errors*
- *Ang WBC og RBC butangi og guide kong unsa dapat e butang since range mana sya, dili na sya integer, pwede raka mag make og option list similar sa imong ge buhat sa other features, pwede raka mag base danhi:*


<code>["0-0", "0-1", "0-2", "1-2", "0-3", "0-4", "1-3", "2-3", "1-4", "2-4", 
                            "1-5", "2-5", "3-4", "1-6", "3-5", "2-6", "3-6", "2-7", "4-5", "3-7",
                            "4-6", "4-7", "5-6", "5-7", "4-8", "3-10", "5-8", "6-8", "4-10","5-10",
                            "7-8", "7-9", "7-10", "8-10", "8-11", "6-14", "8-12", "9-11", "9-12",
                            "7-15", "10-12", "9-15", "11-13", "10-15", "11-14", "10-16", "11-15",
                            "12-14", "12-15", "10-18", "13-15", "12-17", "14-16", "15-17", "15-18",
                            "16-18", "15-20", "15-21", "17-20", "15-22", "18-20", "18-21", "18-22",
                            "20-22", "15-28", "18-25", "20-25", "22-24", "23-25", "25-30", "25-32",
                            "28-30", "30-32", "28-35", "30-35", "34-36", "36-38", "35-40", "38-40",
                            "45-50", ">50", "48-55", "50-55", "48-62", "55-58", "70-75", "79-85",
                            "85-87", ">100", "LOADED", "TNTC"]</code>



### Load the Data

In [18]:
# import the libs
import pandas as pd
import numpy as np

from joblib import load

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm", message="Found boosting=goss")

### Store the users inputs into a dataframe

In [6]:
# example sa user input, exluding and diagnosis
user_input_df = pd.DataFrame()

# user input exammple
user_input_df["Age"] = [76.0]
user_input_df["Gender"] = ["MALE"]
user_input_df["Color"] = ["LIGHT YELLOW"]
user_input_df["Transparency"] = ["CLEAR"]
user_input_df["Glucose"] = ["NEGATIVE"]
user_input_df["Protein"] = ["NEGATIVE"]
user_input_df["pH"] = [5.0]
user_input_df["Specific_Gravity"] = [1.01]
user_input_df["WBC"] = ["1-3"]
user_input_df["RBC"] = ["0-2"]
user_input_df["Epithelial_Cells"] = ["OCCASIONAL"]

user_input_df["Mucous_Threads"] = ["RARE"]
user_input_df["Amorphous_Urates"] = ["NONE SEEN"]
user_input_df["Bacteria"] = ["OCCASIONAL"]

user_input_df


,Age,Gender,Color,Transparency,Glucose,Protein,pH,Specific_Gravity,WBC,RBC,Epithelial_Cells,Mucous_Threads,Amorphous_Urates,Bacteria
0,76.0,MALE,LIGHT YELLOW,CLEAR,NEGATIVE,NEGATIVE,5.0,1.01,1-3,0-2,OCCASIONAL,RARE,NONE SEEN,OCCASIONAL


### Encoding the inputs of the user

In [7]:
def custom_sort(value):
    """
        Sorting string range values, and with greater than or less than, e.g, 80-90, <50, 100> or LOADED

        Parameters
        ------------
        value : str
            the string data that needs to be sorted

        Returns
        ------------
        if ```value``` contains ,>,, return an integer without the sign.
        elif ```value``` is equal to LOADED or TNTC (Too Numerous to Count), return the highest floating point number.
        elif ```value``` is a range e.g 10-15, calculate the midpoint of the range by splitting the range into two number and return the midpoint
    """
    
    if value.__contains__(">"):
        return int(value.replace(">", ""))
    elif value.isalpha():
        return int(np.finfo(np.float64).max)
    else:
        start, end = map(int, value.split("-"))
        return start + (end - start) / 2

def encode_features(dataframe_, ordinal_features, nominal_features):
    """
        Encodes the nominal and ordinal features of the dataset using map function for ordinal and One-Hot Encoding for nominal.

        Parameters
        ------------
        dataframe_ : pandas.core.frame.DataFrame
            the dataframe that contains the nominal and orginal features

        ordinal_features : list, tuple, set
            a list of strings that corresponds to the column name of the ordinal features

        nominal_features : list, tuple, set
            a list of strings that corresponds to the column name of the nominal features

        Returns
        ------------
        dataframe_ : pandas.core.frame.DataFrame
            returns the preprocessed dataframe with all the ordinal and nominal features encoded
    """
    
    
    MAPPING_REFERENCE = {"ABSENCE_REFERENCE": {"NONE SEEN":0, 
                                               "RARE":1,
                                               "FEW":2,
                                               "OCCASIONAL":3,
                                               "MODERATE":4,
                                               "LOADED":5,
                                               "PLENTY":6},
                        
                        "Color": {"LIGHT YELLOW":0,
                                  "STRAW":1,
                                  "AMBER":2,
                                  "BROWN":3,
                                  "DARK YELLOW":4,
                                  "YELLOW":5,
                                  "REDDISH YELLOW":6,
                                  "REDDISH":7,
                                  "LIGHT RED":8,
                                  "RED":9},
                         
                        "Transparency": {"CLEAR":0,
                                         "SLIGHTLY HAZY":1,
                                         "HAZY":2,
                                         "CLOUDY":3,
                                         "TURBID":4},

                        "Protein_and_Glocuse": {"NEGATIVE":0,
                                                "TRACE":1,
                                                "1+":2,
                                                "2+":3,
                                                "3+":4,
                                                "4+":5}}
    
    SORTED_RANGED_VALUES = ["0-0", "0-1", "0-2", "1-2", "0-3", "0-4", "1-3", "2-3", "1-4", "2-4", 
                            "1-5", "2-5", "3-4", "1-6", "3-5", "2-6", "3-6", "2-7", "4-5", "3-7",
                            "4-6", "4-7", "5-6", "5-7", "4-8", "3-10", "5-8", "6-8", "4-10","5-10",
                            "7-8", "7-9", "7-10", "8-10", "8-11", "6-14", "8-12", "9-11", "9-12",
                            "7-15", "10-12", "9-15", "11-13", "10-15", "11-14", "10-16", "11-15",
                            "12-14", "12-15", "10-18", "13-15", "12-17", "14-16", "15-17", "15-18",
                            "16-18", "15-20", "15-21", "17-20", "15-22", "18-20", "18-21", "18-22",
                            "20-22", "15-28", "18-25", "20-25", "22-24", "23-25", "25-30", "25-32",
                            "28-30", "30-32", "28-35", "30-35", "34-36", "36-38", "35-40", "38-40",
                            "45-50", ">50", "48-55", "50-55", "48-62", "55-58", "70-75", "79-85",
                            "85-87", ">100", "LOADED", "TNTC"]
    
    for ordinal in ordinal_features:
        if ordinal in ["Epithelial_Cells", "Mucous_Threads", "Amorphous_Urates", "Bacteria"]:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE["ABSENCE_REFERENCE"])

        elif ordinal in ["Protein", "Glucose"]:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE["Protein_and_Glocuse"])

        elif ordinal in ["WBC", "RBC"]:
            num_of_splits = 13
            splitted_array = np.hsplit(np.array(SORTED_RANGED_VALUES), num_of_splits)

            list_of_bin = [bin_0, bin_1, bin_2, bin_3, bin_4, bin_5, bin_6, bin_7, bin_8, bin_9, bin_10, bin_11, bin_12] = splitted_array
        
            def mapping_function(value):
                map_ = {tuple(bin_):i for i, bin_ in enumerate(list_of_bin)}

                for key, reference in map_.items():
                    if value in key:
                        return reference

            dataframe_[ordinal] = dataframe_[ordinal].map(mapping_function)
        else:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE[ordinal])
            
    for nominal in nominal_features:

        dataframe_["FEMALE"] = [True if dataframe_[nominal].values[0] == "FEMALE" else False]
        dataframe_ = dataframe_.drop(nominal, axis=1)
        
    return dataframe_


In [8]:
encoded_user_input = encode_features(user_input_df, 
                                    ordinal_features=["Transparency", 
                                        "Epithelial_Cells", 
                                        "Mucous_Threads", 
                                        "Amorphous_Urates", 
                                        "Bacteria",
                                        "Color",
                                        "Protein",
                                        "Glucose",
                                        "WBC",
                                        "RBC"],

                                    nominal_features=["Gender"])

In [9]:
encoded_user_input
# user input is encoded and ready to feed to the model for diagnosis

,Age,Color,Transparency,Glucose,Protein,pH,Specific_Gravity,WBC,RBC,Epithelial_Cells,Mucous_Threads,Amorphous_Urates,Bacteria,FEMALE
0,76.0,0,0,0,0,5.0,1.01,0,0,3,1,0,3,False


### Loading the trained model using joblib to make predictions

In [20]:
LGBM_MODEL = load("model/best_model.joblib")

In [22]:
pred = LGBM_MODEL.predict(encoded_user_input)[0]

[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.


In [23]:

print(f"Model Diagnosis: {'UTI Positive' if pred else 'UTI NEGATIVE'}")

Model Diagnosis: UTI NEGATIVE
